# Regressão Linear Múltipla

A Regressão Linear Múltipla é uma extensão da Regressão Linear Simples, usada quando queremos modelar a relação entre uma variável dependente (o que queremos prever) e duas ou mais variáveis independentes (os fatores que usamos para fazer a previsão).

Na implementação da regressão linear simples, criamos um modelo que y = α + βxₙ + εₙ. Agora, imagine que cada entrada xₙ não é um único número, mas um vetor de k números.
O modelo de regressão linear múltipla pressupões que:   
<br>
<br>
$$
y = α + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε

$$
<br>

Cada termo tem um significado importante:

- y --> Variável dependente (o que queremos prever).
- x₁, x₂, ..., xₙ --> Variáveis independentes (o que usamos para fazer a previsão).
- β₁, β₂, ..., βₙ --> Coeficientes que medem o impacto de cada variável independente em y.
- α --> Intercepto, o valor de y quando todas as variáveis independentes são zero.
- ε --> Erro, que representa a variação não explicada pelo modelo.

Na regressão linear múltipla, o vetor de paramêtros geralmente é chamado de β. Também temos que incluir o termo costante, adicionando uma coluna de 1s aos dados:

beta = [alpha, beta1, ..., beta_k]

x_i = [1, x_i1, ..., x_ik]

Nosso modelo é:
```python
def predict(x: Vector, beta: Vector) -> float:
    """Pressupõe que o primeiro elemento de x é 1"""
    return sum(x_i * b for x_i, b in zip(x, beta))
```

## Hipóteses Adicionais do Modelo de Mínimos Quadrados
Para que o modelo funcione corretamente, algumas hipóteses devem ser atendidas:

1. Independência Linear<br>
As variáveis independentes (colunas de X) não devem ser combinações lineares entre si.
- Se uma variável puder ser escrita como uma soma ponderada de outras, não será possível estimar os coeficientes (β).
- Exemplo: Se tivermos duas variáveis idênticas, como num_amigos e num_conhecidos, o modelo não conseguirá diferenciar suas contribuições.

2. Não Correlação com o Erro<br>
As variáveis independentes não devem estar correlacionadas com o erro (ϵ).
- Se essa condição for violada, as estimativas dos coeficientes (β) serão tendenciosas.
- Exemplo: Se pessoas que trabalham mais horas passam menos tempo em um site, mas o modelo não considera as horas de trabalho, o coeficiente de num_amigos será subestimado, pois ignora essa relação indireta.

Em resumo, se as variáveis não forem independentes ou estiverem correlacionadas com o erro, as previsões do modelo serão imprecisas e os coeficientes estimados serão enviesados.

## Ajustando o Modelo

Como fizemos no modelo linear simples, escolheremos o beta para minimizar a soma dos erros quadráticos. Como não é tão simples definir uma solução exata manualmente, temos que usar o gradiente descendente. Mais uma vez, precisamos minimizar a soma dos erros quadrátricos. A função do erro é quase idêntica à utilizada na implementação da regressão linear simples, mas em vez de esperar os paramêtros (alpha e beta), ela recebará um vetor de tamanho arbitrário:

```python
def error(x: Vector, y: float beta: Vector) -> float:
    return predict(x, beta) - y

def squared_error(x: Vector, y: float, beta: Vector) -> float:
    return error(x, y, beta) ** 2

def sqerror_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    err = error(x, y, beta)
    return [2 * err * x_i for x_i in x]
```


Antes de aplicar usar o gradiente, precisamos escrever uma função least_squares_fit que opere em qualquer conjunto de dados:

```python
def least_squares_fit(xs: List[Vector],
                      ys: List[float],
                      learning_rate: float = 0.001,
                      num_steps: int = 1000,
                      batch_size: int = 1) -> Vector:
    """
    Encontre o beta que minimiza a soma dos erros qudráticos
    pressupondo que o modelo y = dot(x, beta)
    """
    # Comece com uma estimativa aleatória
    guess = [random.random() for _ in xs[0]]

    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start+batch_size]
            batch_ys = ys[start:start+batch_size]

            gradient = vector_mean([sqerror_gradient(x, y, guess)
                                    for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess
```

## Implementando

#### Análise de Tempo de Uso de um Aplicativo de Streaming

Queremos entender quanto tempo (em minutos) os usuários passam assistindo a conteúdos em um aplicativo de streaming. As variáveis que influenciam o tempo assistido são:

- Constante: Um valor fixo de 1 para ajustar o modelo.
- Número de dispositivos: Quantos dispositivos o usuário usa para acessar o aplicativo (ex.: 1 a 5).
- Horas de trabalho por dia: Quantas horas o usuário trabalha diariamente (ex.: 4 a 12 horas).
- Assinante premium: Se o usuário é assinante premium ou não (0 para não, 1 para sim).

#### Relação esperada entre as variáveis e o tempo no aplicativo:

- Número de dispositivos: Quanto mais dispositivos, maior a chance de assistir em diferentes momentos (+).
- Horas de trabalho: Quanto mais trabalha, menos tempo tem para assistir (-).
- Assinante premium: Assinantes premium tendem a consumir mais conteúdo (+).

In [1]:
import random

def gerar_dados_streaming(n=200):
    random.seed(42)
    X = []
    y = []
    for _ in range(n):
        constante = 1.0 # Constante para o modelo 
        num_dispositivos = random.randint(1, 5) # Número de dispositivos (entre 1 e 5)
        horas_trabalho = random.randint(4, 12) # Horas de trabalho por dia (entre 4 e 12)
        assinante_premium = random.choice([0, 1]) # Assinante premium (0 = não, 1 = sim)

        # Coeficientes do modelo
        beta_constante = 30  # Base fixa de minutos
        beta_dispositivos = 12  # Mais dispositivos, mais tempo
        beta_horas_trabalho = -3  # Mais trabalho, menos tempo
        beta_premium = 20  # Assinantes assistem mais

        # Gerando o tempo de uso (com ruído aleatório)
        minutos_assistidos = (
            beta_constante
            + beta_dispositivos * num_dispositivos
            + beta_horas_trabalho * horas_trabalho
            + beta_premium * assinante_premium
            + random.uniform(-5, 5)  # Pequeno ruído
        )

        X.append([constante, num_dispositivos, horas_trabalho, assinante_premium])
        y.append(round(minutos_assistidos, 2))

    return X, y

X, y = gerar_dados_streaming()

for i in range(5):
    print(f"Entrada: {X[i]}, Minutos assistidos: {y[i]}")

Entrada: [1.0, 1, 4, 1], Minutos assistidos: 47.45
Entrada: [1.0, 2, 5, 0], Minutos assistidos: 39.9
Entrada: [1.0, 1, 4, 0], Minutos assistidos: 27.19
Entrada: [1.0, 5, 4, 0], Minutos assistidos: 80.16
Entrada: [1.0, 5, 10, 0], Minutos assistidos: 59.49


In [2]:
from reglin_multipla import least_squares_fit

beta = least_squares_fit(X, y, learning_rate=0.0001, num_steps=5000, batch_size=30)
print("Coeficientes encontrados:")
print("Intercepto: ", beta[0])
print("Numero de dispositivos: ", beta[1])
print("Horas de trabalho: ", beta[2])
print("Assinante premium: ", beta[3])

least squares fit: 100%|██████████| 5000/5000 [00:01<00:00, 4279.00it/s]

Coeficientes encontrados:
Intercepto:  13.691553345061315
Numero de dispositivos:  13.668478695946117
Horas de trabalho:  -1.6819805610756267
Assinante premium:  19.561160620687385


Os coeficientes estimados (beta) têm a seguinte interpretação:
- β₀ (Intercepto = 13.69): Representa o tempo médio (em minutos) que os usuários passam no aplicativo quando todas as variáveis independentes são zero. Esse é o valor base.

- β₁ (Número de dispositivos = 13.66): Para cada dispositivo adicional que o usuário possui, o tempo assistindo aumenta em aproximadamente 14 minutos.
- β₂ (Horas de trabalho por dia = -1.68): A cada hora a mais de trabalho, o tempo assistindo no aplicativo diminui cerca de 1.68 minutos.
- β₃ (Assinante premium = 19.56): Usuários com assinatura premium passam, em média, 19.56 minutos a mais no aplicativo em comparação com usuários sem assinatura.

Com esses valores, podemos realizar previsões:

In [ ]:
from reglin_multipla import predict

novos_dados = [1.0, 3, 6, 1]
predict(novos_dados, beta)

64.16626668713329

Como na regressão linear simples, aqui também usaremos o R-quadrado:

In [4]:
from reglin_multipla import multiple_r_squared

multiple_r_squared(X, y, beta)

0.935108789613937

## Scikit-Learn

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

reg_lin_sklearn = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

reg_lin_sklearn.fit(X_train, y_train)

pred = reg_lin_sklearn.predict(X_test)
r2_score(y_test, pred)

0.9808567694310053

In [8]:
reg_lin_sklearn.predict([novos_dados])

array([68.09873175])

Esse é um valor bastante próximo da implementação feita do zero! (64.16)

In [6]:
reg_lin_sklearn.intercept_, reg_lin_sklearn.coef_

(np.float64(29.592446296751408),
 array([ 0.        , 12.07239782, -3.00280504, 20.30592222]))

- β₀ (Intercepto = 29.59): Este é o valor de base do tempo de uso do aplicativo quando todas as outras variáveis são zero. Isso sugere que, com o modelo do scikit-learn, o tempo médio inicial de uso (quando todas as outras variáveis são zero) é de aproximadamente 29.59 minutos.

- β₁ (Número de dispositivos = 12.07): Esse coeficiente indica que, para cada dispositivo adicional que o usuário possui, o tempo de uso do aplicativo aumenta em aproximadamente 12.07 minutos. Ou seja, usuários que possuem mais dispositivos tendem a usar o aplicativo por mais tempo, o que pode indicar que eles estão acessando o aplicativo de diferentes plataformas.
- β₂ (Horas de trabalho por dia = -3.00): O coeficiente negativo de -3.00 sugere que, para cada hora adicional de trabalho, o tempo de uso do aplicativo diminui em 3 minutos, o que é um pouco mais acentuado em comparação com o modelo do zero (-1.71).
- β₃ (Assinante premium = 20.30): A diferença de tempo entre usuários com e sem assinatura premium é de 20.30 minutos, o que é muito próximo do valor do modelo do zero (19.56 minutos).

Os coeficientes dos modelos são bastante próximos, mas há uma grande diferença no intercepto. O modelo do scikit-learn estima um valor de 29.59 minutos para o tempo base, enquanto o modelo do zero estima 13.69 minutos. Em relação às variáveis, a diferença no número de dispositivos (12.07 vs 13.66) é pequena, indicando um impacto semelhante. O modelo do scikit-learn tem uma maior sensibilidade às horas de trabalho (-3.00 vs -1.68), e ambos concordam que a assinatura premium aumenta o tempo de uso, com uma diferença de 1 minuto entre eles.